<a href="https://colab.research.google.com/github/artemisak/twitter_emotion_clf/blob/main/advansed_sentiments_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers[torch] scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 17.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (4

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("carblacac/twitter-sentiment-analysis", trust_remote_code=True)

# Возьмем маленькую часть этих датасетов, потому что у меня нет 750 часов для обучения этих моделей)
train_dataset = dataset["train"].train_test_split(test_size=0.8)
validation_dataset = dataset["validation"].train_test_split(test_size=0.8)
test_dataset = dataset["test"].train_test_split(test_size=0.8)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    # Опять же настроим так, чтобы сократить время обучения
    return tokenizer(examples["text"], padding=True, truncation=True, max_length=64)

tokenized_train_dataset = train_dataset["train"].map(tokenize_function, batched=True)
tokenized_train_dataset = tokenized_train_dataset.rename_column("feeling", "labels")
tokenized_train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

tokenized_validation_dataset = validation_dataset["validation"].map(tokenize_function, batched=True)
tokenized_validation_dataset = tokenized_validation_dataset.rename_column("feeling", "labels")
tokenized_validation_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

tokenized_test_dataset = test_dataset["test"].map(tokenize_function, batched=True)
tokenized_test_dataset = tokenized_test_dataset.rename_column("feeling", "labels")
tokenized_test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/149985 [00:00<?, ? examples/s]

Map:   0%|          | 0/61998 [00:00<?, ? examples/s]

...

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/23997 [00:00<?, ? examples/s]

Map:   0%|          | 0/49599 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import classification_report
import numpy as np

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=128,
    num_train_epochs=3,
    weight_decay=0.01,
)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return {"classification_report": classification_report(labels, predictions, output_dict=True)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Classification Report
1,0.461300,0.403393,"{'0': {'precision': 0.8150153258230166, 'recall': 0.8260641517046601, 'f1-score': 0.8205025447841943, 'support': 24785}, '1': {'precision': 0.8238826701527903, 'recall': 0.8127266865479165, 'f1-score': 0.8182666558467905, 'support': 24814}, 'accuracy': 0.8193915199903223, 'macro avg': {'precision': 0.8194489979879034, 'recall': 0.8193954191262882, 'f1-score': 0.8193846003154923, 'support': 49599}, 'weighted avg': {'precision': 0.8194515903081677, 'recall': 0.8193915199903223, 'f1-score': 0.819383946665427, 'support': 49599}}"


Trainer is attempting to log a value of "{'0': {'precision': 0.8150153258230166, 'recall': 0.8260641517046601, 'f1-score': 0.8205025447841943, 'support': 24785}, '1': {'precision': 0.8238826701527903, 'recall': 0.8127266865479165, 'f1-score': 0.8182666558467905, 'support': 24814}, 'accuracy': 0.8193915199903223, 'macro avg': {'precision': 0.8194489979879034, 'recall': 0.8193954191262882, 'f1-score': 0.8193846003154923, 'support': 49599}, 'weighted avg': {'precision': 0.8194515903081677, 'recall': 0.8193915199903223, 'f1-score': 0.819383946665427, 'support': 49599}}" of type <class 'dict'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

predictions, labels, _ = trainer.predict(tokenized_test_dataset)
predictions = np.argmax(predictions, axis=1)
print(classification_report(labels, predictions))